In [3]:
import ipywidgets as widgets
from IPython.display import display, HTML
import pandas as pd

# 1. CSS for complete background coverage and consistent #F4F4F4 accents
display(HTML("""
<style>
    .main-container {
        background-color: #1E3860 !important;
        padding: 30px;
        border-radius: 12px;
    }
    
    .widget-vbox, .p-Accordion-child { 
        background-color: #1E3860 !important; 
    }
    
    /* Labels: Increased size and spacing */
    .widget-label { 
        color: #F4F4F4 !important; 
        font-weight: bold; 
        font-size: 14px !important;
        width: 250px !important; 
    }
    
    /* Input Boxes: Increased padding and font size */
    .widget-text input, .widget-dropdown select, .widget-floattext input { 
        background-color: #F4F4F4 !important; 
        color: #1E3860 !important;
        border: 1px solid #F4F4F4 !important;
        font-size: 14px !important;
        padding: 5px 10px !important;
        height: auto !important;
    }
    
    /* Ensuring the dropdown select element itself allows for flexible width */
    .widget-dropdown select {
        width: auto !important;
        min-width: 200px;
    }

    .p-Accordion-header {
        background-color: #264b80 !important;
        color: #F4F4F4 !important;
        font-size: 16px !important;
    }

    .submit-btn { 
        background-color: #F4F4F4 !important; 
        color: #1E3860 !important; 
        font-weight: 800 !important; 
        height: 50px !important;
        font-size: 16px !important;
        margin-top: 20px;
    }
</style>
"""))


In [4]:

def deal_entry_ui():
    # --- SECTION 1: RELATIONSHIP SUMMARY ---
    rel_account = widgets.Text(description='Relationship Account:')
    deal_name = widgets.Text(description='New Deal Name:')
    cost_center = widgets.Dropdown(
        options=[('1005 (HOA)', '1005')
                 , ('1090 (BB DC)', '1090')
                 , ('1091 (MM AZ)', '1091')
                 , ('1092 (MM AZ)','1092')
                 , ('1100 (Equipment Finance)','1100')
                 , ('1110 (Commercial Client Lending)','1110')
                 , ('1111 (Sponsored Finance)','1111')
                 ],
        description='Cost Center:'
    )
    category = widgets.Dropdown(
        options=['CRE', 'C&I', 'SFR', 'Constructions', 'Others'],
        description='Category:'
    )
    product = widgets.Dropdown(
        options=['Revolving Line of Credit', 'Deposit Only', 'Term Loan'],
        description='Product:'
    )
    security = widgets.Dropdown(
        options=['Cash Secured', 'Cross Collaterized', 'Senior Secured', 'Senior Unsecured', '2nd lien', 'Mixed - 1st & 2nd lien'],
        description='Security:'
    )
    risk_rating = widgets.Text(description='Proposed Risk Rating:')
    committed = widgets.Dropdown(options=['Yes', 'No'], description='Committed:')
    cap_modifier = widgets.Dropdown(
        options=['N/A - No', 'Security/SSFA/NDFI', 'HVCRE - Classified', 'CRA - Qualifying'],
        description='Capital Treatment Modifier:'
    )

    rel_summary_box = widgets.VBox([
        rel_account, deal_name, cost_center, category, 
        product, security, risk_rating, committed, cap_modifier
    ])

    # --- SECTION 2: LOAN LEVEL INPUTS ---
    spread = widgets.FloatText(description='Spread (%):', value=0.00, step=0.01)
    curr_index = widgets.FloatText(description='Current Index (%):', value=0.00, step=0.01)
    index_floor = widgets.FloatText(description='Index Floor (%):', value=0.00, step=0.01)
    all_in_floor = widgets.FloatText(description='All-in Floor Rate (%):', value=0.00, disabled=True)
    
    commitment = widgets.FloatText(description='Total Commitment:')
    funding_sched = widgets.Text(description='Funding Schedule:')
    amt_funded = widgets.FloatText(description='Amount Funded at Close:')
    first_month = widgets.Text(description='First Month (YYYY-MM):')

    loan_level_box = widgets.VBox([
        spread, curr_index, index_floor, all_in_floor, 
        commitment, funding_sched, amt_funded, first_month
    ])

    # --- CALCULATION LOGIC ---
    def update_floor(change):
        all_in_floor.value = round(spread.value + index_floor.value, 2)

    spread.observe(update_floor, names='value')
    index_floor.observe(update_floor, names='value')

    # --- ASSEMBLY ---
    accordion = widgets.Accordion(children=[rel_summary_box, loan_level_box])
    accordion.set_title(0, 'Relationship Summary')
    accordion.set_title(1, 'Loan Level Inputs')
    
    submit_button = widgets.Button(description="PROCESS DEAL ENTRY", layout=widgets.Layout(width='100%'))
    submit_button.add_class("submit-btn")
    
    output = widgets.Output()

    def on_submit(b):
        with output:
            output.clear_output()
            # Compile logic for your Python dataframe
            res = {"Account": rel_account.value, "Spread": spread.value, "Total Floor": all_in_floor.value}
            print("✅ Deal Captured:")
            display(pd.DataFrame([res]))

    submit_button.on_click(on_submit)

    # Wrap everything in a main container to force the background color
    main_page = widgets.VBox([accordion, submit_button, output])
    main_page.add_class("main-container")
    
    display(main_page)

deal_entry_ui()